Notebook for analyzing the LAMMPS output for the sea fans. Requires the output of the equilibration simulation (.xyz or .lammpstrj file) and a .lammpstrj and MSD file from the final LAMMPS simulation (alternatively, only the .lammpstrj file can be read in and MSD calculated by hand).   

By: Ariana Vlad

# Imports

In [ ]:
import csv
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from colour import Color
import sys
import os
import random

# Create LAMMPS Input Files From Raw Data

## Read the raw data

In [ ]:
data=pd.read_csv('tree_skeletonized.csv')
#data=pd.read_csv('network_skeletonized.csv')

x=data.iloc[1:len(data),0]
y=data.iloc[1:len(data),2]

## Create a LAMMPS input file for equilibration

In [ ]:
#create a contact matrix to hold the bonds and angles
#the matrix's ij entry is 1 if the ith and jth atom are bonded, and 0 otherwise

r = 3 #depends on the raw data, usually overestimating is better to be sure all segments are connected

contact_matrix=np.zeros((len(x), len(x)), dtype=int)

for i in range(len(x)):
    for j in range(len(x)):
        if math.sqrt((float(y.iloc[i])-float(y.iloc[j]))**2+(float(x.iloc[i])-float(x.iloc[j]))**2)<r:
            contact_matrix[i][j]=1


#calculate the total number of bonds and angles

bonds=0

for i in range(len(x)):
    for j in range(i):
        if contact_matrix[i][j]==1:
            bonds+=1

angles=0

for i in range(len(x)):
    for j in range(len(x)):
        if contact_matrix[i][j]==1 and j!=i:
            for k in range(i):
                if contact_matrix[j][k]==1 and j!=k:
                    angles+=1

In [ ]:
with open('tree_equilibration.txt', 'w') as f:
  sys.stdout=f
  f.write('LAMMPS data file for tree - equilibration \n \n')
  f.write(len(x), ' atoms \n')
  f.write(bonds, ' bonds \n')
  f.write(angles, ' angles \n')
  f.write('\n 1 atom types \n 1 bond types \n 1 angle types \n \n -100 100 xlo xhi \n - 100 100 ylo yhi \n -100 100 zlo zhi \n \n Masses \n 1 1 \n \n')
  f.write('Atoms \n \n')

  for i in range(len(x)):
    f.write(i+1, ' ', 1, ' ', 1, ' ', float(x.iloc[i]), ' ', float(y.iloc[i]), ' ', 0)

  f.write('\n \n Bonds \n \n ')

  count=0
  for i in range(len(x)):
    for j in range(i):
      if contact_matrix[i][j]==1:
        count+=1
          f.write(count, ' ', 1, ' ', i+1, ' ',   j+1)

  f.write('\n \n Angles \n \n ')

  count=0
  for i in range(len(x)):
    for j in range(len(x)):
      if contact_matrix[i][j]==1 and j!=i:
        for k in range(i):
          if contact_matrix[j][k]==1 and j!=k:
            count+=1
              f.write(count, ' ', 1, ' ', i+1, ' ',   j+1, ' ', k+1)

## Check Equilibration

In [ ]:
#open output of equilibration

with open("tree_equilibration.xyz") as file:
	lines=[]
	for line in file:
		lines.append(line)

x_eq=[]
y_eq=[]
z_eq=[]

for i in range(len(lines)):
    if lines[i]=='Atoms. Timestep: 100000\n':
        for j in range(len(x)):
          x_eq.append(float(lines[i+j+1].split(' ')[1]))
          y_eq.append(float(lines[i+j+1].split(' ')[2]))
          z_eq.append(float(lines[i+j+1].split(' ')[3]))

#plot the histogram of the bonds length

d=[]

for i in range(len(x_eq)):
	for j in range(i):
		if contact_matrix[i][j]==1:
			d.append(math.sqrt((x_eq[i]-x_eq[j])**2+(y_eq[i]-y_eq[j])**2+(z_eq[i]-z_eq[j])**2))

plt.hist(d)
plt.xlabel('Bond Length')
plt.ylabel('Count')

#if the optimization was sucessful, the distribution is narrowly peaked at 1

## Create the new LAMMPS input file

In [ ]:
#create a contact matrix to hold the bonds and angles
#the matrix's ij entry is 1 if the ith and jth atom are bonded, and 0 otherwise

r = 1.5 #between 1 and 2 to connect all segments and account for the variation from equilibration

contact_matrix=np.zeros((len(x_eq), len(x_eq)), dtype=int)

for i in range(len(x_eq)):
    for j in range(len(x_eq)):
        if math.sqrt((x_eq[i]-x_eq[j])**2+(y_eq[i]-y_eq[j])**2+(z_eq[i]-z_eq[j])**2)<r:
            contact_matrix[i][j]=1

#calculate the total number of bonds and angles

bonds=0

for i in range(len(x_eq)):
    for j in range(i):
        if contact_matrix[i][j]==1:
            bonds+=1

angles=0

for i in range(len(x_eq)):
    for j in range(len(x_eq)):
        if contact_matrix[i][j]==1 and j!=i:
            for k in range(i):
                if contact_matrix[j][k]==1 and j!=k:
                    angles+=1

In [ ]:
with open('tree.txt', 'w') as f:
  sys.stdout=f
  f.write('LAMMPS data file for tree \n \n')
  f.write(len(x), ' atoms \n')
  f.write(bonds, ' bonds \n')
  f.write(angles, ' angles \n')
  f.write('\n 1 atom types \n 1 bond types \n 1 angle types \n \n -100 100 xlo xhi \n - 100 100 ylo yhi \n -100 100 zlo zhi \n \n Masses \n 1 1 \n \n')
  f.write('Atoms \n \n')

  for i in range(len(x_eq)):
    f.write(i+1, ' ', 1, ' ', 1, ' ', x_eq[i], ' ', y_eq[i], ' ', z_eq[i])

  f.write('\n \n Bonds \n \n ')

  count=0
  for i in range(len(x_eq)):
    for j in range(i):
      if contact_matrix[i][j]==1:
        count+=1
          f.write(count, ' ', 1, ' ', i+1, ' ',   j+1)

  f.write('\n \n Angles \n \n ')

  count=0
  for i in range(len(x_eq)):
    for j in range(len(x_eq)):
      if contact_matrix[i][j]==1 and j!=i:
        for k in range(i):
          if contact_matrix[j][k]==1 and j!=k:
            count+=1
            f.write(count, ' ', 1, ' ', i+1, ' ',   j+1, ' ', k+1)

# Analyze Output

## Mean square displacement (time)

In [ ]:
seeds=['123', '34623', '2781115']
timesteps=100001

tree_msd=np.zeros((len(seeds), timesteps))

for i in range(len(seeds)):
  with open('tree_msd_10_'+seeds[i]+'.txt') as file:
    lines=[]
    for line in file:
      lines.append(line)

  for j in range(timesteps):
    tree_msd[i][j]=float(lines[j+2].split(' ')[1])

In [ ]:
mean_msd=np.mean(tree_msd, 0)
sd_msd=np.std(tree_msd, 0)

plt.plot(range(1, timesteps+1), tree_msd[0], label='seed = 123', color=0.8)
plt.plot(range(1, timesteps+1), tree_msd[1], label='seed = 34623', color=0.8)
plt.plot(range(1, timesteps+1), tree_msd[2], label='seed = 2781115', color=0.8)
plt.errorbar(range(1, timesteps+1), mean_msd, yerr=sd_msd, fmt ='o', label='mean tree')
plt.legend()
plt.xlabel('Timestep')
plt.ylabel('Mean Square Displacement (sigma)')

## Mean square displacement (angle coefficient)

In [4]:
seeds=['123', '34623', '2781115']
angle_coeffs=['0', '10', '25', '50']
timesteps=100001

tree_msd=np.zeros((len(angle_coeffs), len(seeds), timesteps))

for k in range(len(angle_coeffs)):
  for i in range(len(seeds)):
    with open('tree_msd_'+angle_coeffs[k]+'_'+seeds[i]+'.txt') as file:
      lines=[]
      for line in file:
        lines.append(line)

    for j in range(timesteps):
      tree_msd[k][i][j]=float(lines[j+2].split(' ')[1])

In [ ]:
mean_msd=[]
sd_msd=[]

mean_msd=[np.mean(np.mean(tree_msd[i], 0)) for i in range(len(angle_coeffs))]
sd_msd=[np.std(np.mean(tree_msd[i], 0)) for i in range(len(angle_coeffs))]

plt.errorbar(angle_coeffs, mean_msd, yerr=sd_msd, fmt ='o')
plt.xlabel('Angle Coefficient')
plt.ylabel('Mean Square Displacement (sigma)')

## Most dynamics atoms and Maximum displacement per atom

In [ ]:
seeds=['123', '34623', '2781115']
angle_coeffs=['0', '10', '25', '50']
timesteps=100001
atoms=1939

mean_angle=[]
sd_angle=[]

for i in range(len(angle_coeffs)):
  temp_mean=[]
  for j in range(len(seeds)):

    x=[]
    y=[]
    z=[]

    with open('tree_'+angle_coeffs[i]+'_'+seeds[i]+'.lammpstrj') as file:
      lines=[]
      for line in file:
        lines.append(line)

    count=0
    while count<len(lines):
      if lines[count]=='ITEM: ATOMS id x y z ix iy iz type\n':
        x.append([])
        y.append([])
        z.append([])
        for j in range(atoms):
          x[-1].append(float(lines[count+j+1].split(' ')[1]))
          y[-1].append(float(lines[count+j+1].split(' ')[2]))
          z[-1].append(float(lines[count+j+1].split(' ')[3]))
      count+=atoms

    max_displacements=[]  #store the maximum displacement (between consecutive steps) of each atom

    for k in range(len(x)):
      max_displacements.append(max([math.sqrt((x[l][k]-x[l+1][k])**2+(y[l][k]-y[l+1][k])**2+(z[l][k]-z[l+1][k])**2) for l in range(timesteps-1)]))

    #color plot of each atom's maximum displacement
    #points=plt.scatter(x[0], y[0], c=max_displacements/max(max_displacements))
    #plt.colorbar(points)
    #plt.xlabel('x Coordinate')
    #plt.ylabel('y Coordinate')

    #select the 20% most dynamic particles in this simulation

    max_displacements.sort(reverse=True)
    temp_mean.append(np.mean(max_displacements[0:int(0.2*atoms):1]))

  mean_angles.append(np.mean(temp_mean))
  sd_angles.append(np.std(temp_mean))

In [ ]:
plt.errorbar(angle_coeffs, mean_angles, yerr=sd_angles, fmt ='o')
plt.xlabel('Angle Coefficient')
plt.ylabel('Maximum Displacements (sigma)')